**Influence de la crédibilité de la news sur p**

In [1]:
#numpy.random.choice(liste(ou entier) , taille de l'array , replace=False pour pas avoir deux fois la meme valeur , proba)

import numpy as np
import random as rd


news = np.random.random_sample()
print('Crédibilité de l\'info:',news)

def influ_credib_news(news,p):
    #news: crédibilité de la news
    #p: param déterminant le chgt ou non de l'opinion    
    i = p
    i = i+news*(news-i)
    return i

print(influ_credib_news(0.8,0.5))

Crédibilité de l'info: 0.08914771909282393
0.74


**Influence du voisinage sur p**

In [18]:
N=15 #nombre de membres du réseau discussion physique
#description d'un agent : (nom,opinion,deja_vu,persuasion,chgt_opinion)
Reseau_discu=[(str(k),0,False,0.5,0.5) for k in range(N)]
Noms_reseau_discu=[str(k) for k in range(N)]

size = int(np.random.choice(range(2,10)))
Discu = np.random.choice(Noms_reseau_discu,size,replace=False)
print('Membres de la discussion',Discu)

def matrice_discu(L):
    matrice=np.matrix([['00','00','00'],['00','00','00'],['00','00','00']])
    i=0
    j=0
    for (nom,opinion,deja_vu,persu,p) in Reseau_discu:
        if nom in L:
            matrice[i,j]=nom
            if j!=2:
                j=j+1
            elif j==2:
                j=0
                i=i+1
    return matrice
    
print(matrice_discu(Discu.copy()))

Test_reseau=[(str(k),np.random.choice([0,1]),np.random.choice([False,True]),np.random.random_sample(),np.random.random_sample()) for k in range(N)]
print('Réseau aléatoire :',Test_reseau)

def influ_voisinage(discussion,reseau):
    
    P=[]    #liste des p(conviction) de la discussion
    for k in discussion:
        for (nom,opinion,deja_vu,persu,p) in reseau:
            if k == nom:
                if deja_vu:
                    P.append(p)
    plus_convaincu=max(P)     #p du plus convaincu
    moins_convaicnu=min(P)    #p du moins convaincu
    pi=0.5                    #current conviction
    Pi=[]                     #list of the current convictions
    for k in discussion:
        for (nom,opinion,deja_vu,persu,p) in reseau:
            if k == nom:
                pi=p
                if pi<plus_convaincu:
                    pi=pi+((plus_convaincu-pi)/2)
                if pi>moins_convaincu:
                    pi=pi+((moins_convaincu-pi)/2)
                Pi.append(nom,pi)
    for (name,pi) in Pi:
        for (nom,opinion,deja_vu,persu,p) in reseau:
            if name == nom:
                p=pi
    return reseau

print(influ_voisinage(matrice_discu(Discu.copy()),Test_reseau))

Membres de la discussion ['11' '0' '9' '10' '12' '8' '3']
[['0' '3' '8']
 ['9' '10' '11']
 ['12' '00' '00']]
Réseau aléatoire : [('0', 1, False, 0.25014500621656777, 0.6630857071490432), ('1', 1, False, 0.44552510234153175, 0.556640925039278), ('2', 0, False, 0.3695236403311023, 0.9682912008078888), ('3', 1, True, 0.21754757037119332, 0.21573458126224898), ('4', 0, False, 0.7081671300955815, 0.6294567679641961), ('5', 0, True, 0.7723391028402312, 0.8592560403267031), ('6', 1, True, 0.12558748605478565, 0.4656572135597321), ('7', 0, True, 0.6229858114124766, 0.8412696850934159), ('8', 1, True, 0.8563649265646917, 0.05256577219205272), ('9', 1, False, 0.6493452515750189, 0.3572445888092771), ('10', 0, True, 0.14219867439156664, 0.6699813033389755), ('11', 0, True, 0.38337293204374945, 0.5688628536959365), ('12', 1, True, 0.08934566993950455, 0.7722575974756065), ('13', 1, False, 0.8213548454017232, 0.5730156155627216), ('14', 1, True, 0.64448165478676, 0.03760509159603864)]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()